# Week 8 exercise

In [1]:
"""
cd C:/Users/thoma/Desktop/Social_data_analyse/socialdata2024
cd C:/Users/thoma/Desktop/Social_data_analyse/Opgaver
"""

'\ncd C:/Users/thoma/Desktop/Social_data_analyse/socialdata2024\ncd C:/Users/thoma/Desktop/Social_data_analyse/Opgaver\n'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
import folium
from folium import plugins
from folium.plugins import HeatMap


In [3]:
# Load data
df = pd.read_csv("C:/Users/thoma/Desktop/Social_data_analyse/Police_Department_Incident_Reports.csv")

In [4]:
# Describe the data

df.head()
df.shape
df.info()
df.describe()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129525 entries, 0 to 2129524
Data columns (total 35 columns):
 #   Column                                                    Dtype  
---  ------                                                    -----  
 0   PdId                                                      int64  
 1   IncidntNum                                                int64  
 2   Incident Code                                             int64  
 3   Category                                                  object 
 4   Descript                                                  object 
 5   DayOfWeek                                                 object 
 6   Date                                                      object 
 7   Time                                                      object 
 8   PdDistrict                                                object 
 9   Resolution                                                object 
 10  Address                       

PdId                                                              0
IncidntNum                                                        0
Incident Code                                                     0
Category                                                          0
Descript                                                          0
DayOfWeek                                                         0
Date                                                              0
Time                                                              0
PdDistrict                                                        1
Resolution                                                        0
Address                                                           0
X                                                                 0
Y                                                                 0
location                                                          0
SF Find Neighborhoods 2 2                       

In [5]:
# Extracting the year
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
# Remove 2018 as this year is not complete
df = df[df['Date'].dt.year != 2018]
df['Year'] = df['Date'].dt.year

In [14]:
# Calculate the number of rows and columns
nrows = math.ceil(len(NCategories) / 5)
ncols = 5

fig = make_subplots(rows=nrows, cols=ncols, subplot_titles=NCategories)

for i, category in enumerate(NCategories, start=1):
    category_df = df[df['Category'] == category]
    bar_data = category_df['Year'].value_counts().reset_index()
    bar = go.Bar(x=bar_data['index'], y=bar_data['Year'], name=category)
    fig.add_trace(bar, row=(i-1)//ncols+1, col=(i-1)%ncols+1)

fig.update_layout(height=300*nrows, showlegend=False)

for i in range(1, len(NCategories) + 1):
    fig.update_xaxes(title_text="Year", row=(i-1)//ncols+1, col=(i-1)%ncols+1)
    fig.update_yaxes(row=(i-1)//ncols+1, col=(i-1)%ncols+1)

fig.show()

We want to look at forgery/counterfeiting. Because this category seems to be decreasing rapidly in recent years. This could be due to the fact that documents are being digitalized more and more and therefore, these are not as easily forged as they were in the past. 

We make a map of SF to show where the forgery/counterfeiting takes place.

In [7]:
randomdata = {
    'CENTRAL': 0.283805288999638,
    'SOUTHERN': 0.8882636532075772,
    'BAYVIEW': 0.45059924801053985,
    'MISSION': 0.6000904430914474,
    'PARK': 0.6362552416309091,
    'RICHMOND': 0.3371857964893169,
    'INGLESIDE': 0.09876749056377487,
    'TARAVAL': 0.009436215026031758,
    'NORTHERN': 0.44884916837512767,
    'TENDERLOIN': 0.06616710190569974
}

In [8]:
import requests
import plotly.express as px
import json

url = "https://raw.githubusercontent.com/suneman/socialdata2022/main/files/sfpd.geojson"
response = requests.get(url)

# Ensure the request was successful
if response.status_code == 200:
    with open("sfpd.geojson", 'wb') as f:
        f.write(response.content)
else:
    print("Failed to download the file.")

# Open the GeoJSON file and load it into a GeoJSON object
with open("sfpd.geojson") as f:
    geojson = json.load(f)

# Convert your GeoJSON object to a DataFrame
dfgeo = pd.json_normalize(geojson['features'])


In [9]:
# Crime category
forgery = df[df['Category'] == 'FORGERY/COUNTERFEITING']

# Aggregate data by police district
district_df = forgery.groupby(['PdDistrict'])['Category'].count()
district_df = district_df.to_frame()
district_df.reset_index(inplace=True)
district_df.rename(columns={'PdDistrict': 'Count'}, inplace=True)
print(district_df)

        Count  Category
0     BAYVIEW      1669
1     CENTRAL      2541
2   INGLESIDE      2094
3     MISSION      2764
4    NORTHERN      2808
5        PARK      1045
6    RICHMOND      1355
7    SOUTHERN      5033
8     TARAVAL      2262
9  TENDERLOIN      1229


In [10]:
fig = px.choropleth_mapbox(district_df, geojson=geojson, locations='Count', color='Category',
                           color_continuous_scale="Bluered",
                           mapbox_style="carto-positron",
                           zoom=11, center = {"lat": 37.77, "lon": -122.4},
                           opacity=0.5,
                           labels={'PdDistrict':'District'}
                          )

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

Here we see that the most forgeries take place in the Southern district. This also makes sense as the financial district of San Francisco is located in the Southern district of San Francisco (https://en.wikipedia.org/wiki/Financial_District,_San_Francisco). Therefore, forged or counterfeited documents would most likely be found here.

In [11]:
import folium
from folium import plugins
from folium.plugins import HeatMap
import pandas as pd
from folium.plugins import HeatMapWithTime


# Filter for 'FORGERY/COUNTERFEITING' category
forgery_df = df[df['Category'] == 'FORGERY/COUNTERFEITING']

# Convert Date column to datetime
forgery_df['Date'] = pd.to_datetime(forgery_df['Date'])

heat_data = []
for month, month_df in forgery_df.groupby(forgery_df['Date'].dt.to_period('M')):
    heat_data.append(
        [[row['Y'], row['X']] for index, row in month_df.iterrows()]
    )


mymap = folium.Map(location=[37.77919, -122.41914], zoom_start=13)


hm = HeatMapWithTime(heat_data, auto_play=True, max_opacity=0.8)
hm.add_to(mymap)

# Display the map
mymap

C:\Users\thoma\AppData\Local\Temp\ipykernel_23048\3012515960.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



From this heatmap we clearly see a decrease in occurences of forgery/counterfeiting over time. We see that the southern district has the highest denisty over time. From this map we can also see that the occurences of forgery/counterfeiting in the southern district are in fact happening in the financial area which can be seen on this map: (https://commons.wikimedia.org/wiki/File:SF_Financial_District_map.png)